## Load Libraries

In [1]:
import pandas as pd
import numpy as np
import time
import pickle
from functions import training_dataset, PrepareWordLists, selectdata, readdata, cleandatastructure, buildwordsindex, trainingstructuredata, getlistofwords, Create_training_data

from sklearn.dummy import DummyClassifier
from sklearn.metrics import (r2_score, mean_squared_error, accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, precision_recall_curve, make_scorer,
                             confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score


## Import Source Data

In [12]:
#ISM_SourceData = readdata()
#with open('ISM_SourceData.dataframe', 'wb') as f:
#    pickle.dump(ISM_SourceData, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('ISM_SourceData.dataframe', 'rb') as f:
    ISM_SourceData = pickle.load(f)
ISM_SD_TestRange = selectdata(ISM_SourceData, "2021-10-15", "2021-11-15")
ISM_SD_TestRange 
country_list = ['Mexico']
ISM_SD_TestRange = cleandatastructure(ISM_SD_TestRange,country_list )

C:\Users\mamoralessr\Documents\GitHubRepo\MarioMorales-CEMEX-FinalProject\functions.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ISM_SourceData_test['FULLDESC'] = ISM_SourceData_test['TITLE']+ ' ' +ISM_SourceData_test['LONGDESCRIPTION']
C:\Users\mamoralessr\Anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\mamoralessr\Documents\GitHubRepo\MarioMorales-CEMEX-FinalProject\functions.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [13]:
ISM_SD_TestRange.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14033 entries, 29 to 428978
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   TICKETID               14033 non-null  object
 1   FULLDESC               14033 non-null  object
 2   AffectedPersonCountry  14033 non-null  object
 3   OWNERGROUP             14033 non-null  object
dtypes: object(4)
memory usage: 548.2+ KB


In [5]:
## Create Set of Data
ISM_test_set1=ISM_SD_TestRange[0:4999]
ISM_test_set2=ISM_SD_TestRange[5000:9999]
ISM_test_set3=ISM_SD_TestRange[10000:14033]
## Create Data Sets (if requrire partition for performance)
#datasets1_2_3 = [ISM_test_set1,ISM_test_set2,ISM_test_set3]
datasets1 = [ISM_test_set1]
datasets2 = [ISM_test_set2]
datasets3 = [ISM_test_set3]

In [35]:
ISM_test_set1.iloc[1500]['FULLDESC']

'SOPORTE GENERAL - CARGA MASIVA \xa0 @KAREN POR FAVOR NOS\\NPUEDES APOYAR A SOLICITAR CAMBIOS MASIVOS DEL CAMPO DE GESTIÓN DE TIEMPOS DEL\\NPERSONAL ANEXO? ACTUALMENTE TIENEN UN 1 Y DEBERÍAN TENER UN 0 (SIN EVALUACIÓN)\\NYA QUE ESTAMOS REALIZANDO UNAS MODIFICACIONES EN LA FORMA DE GENERAR LA NOMINA\\NDE ESTOS EMPLEADOS. \\N\\N\\N\\N \xa0 \\N\\N\\N\\N QUEDAMOS ATENTOS A TUS COMENTARIOS. \\N\\N\\N\\N \xa0 \\N\\N\\N\\N SALUDOS Y GRACIAS. \\N\\N\\N\\N LUIS \\N\\N'

## Create WordList by Group

In [6]:
## Create Dataframe Structure 
ISM_wgwords = pd.DataFrame(columns=['COUNTRY', 'OWNERGROUP', 'WORDS'])
## Process Dataset to create Wordlists
## Could be several process to reduce performance issues and finally consolidate on ISM_wgwords_all
ISM_wgwords1 = PrepareWordLists(datasets1, ISM_wgwords)

2022-04-02 16:33:17.127902
1
2022-04-02 16:33:25.344569
501
2022-04-02 16:33:35.007482
1001
2022-04-02 16:33:44.043655
1501
2022-04-02 16:33:52.813338
2001
2022-04-02 16:34:02.881507
2501
2022-04-02 16:34:14.390623
3001
2022-04-02 16:34:31.232221
3501
2022-04-02 16:34:49.722389
4001
2022-04-02 16:35:07.497431
4501
2022-04-02 16:35:27.362113
Words:  11856  Groups:  127  Countries: 1


In [7]:
time.sleep(15)
ISM_wgwords2 = PrepareWordLists(datasets2, ISM_wgwords)

2022-04-02 16:35:52.674849
1
2022-04-02 16:36:01.299277
501
2022-04-02 16:36:11.184050
1001
2022-04-02 16:36:20.537976
1501
2022-04-02 16:36:30.687216
2001
2022-04-02 16:36:41.784789
2501
2022-04-02 16:36:54.964697
3001
2022-04-02 16:37:10.323901
3501
2022-04-02 16:37:27.162456
4001
2022-04-02 16:37:44.920278
4501
2022-04-02 16:38:06.451083
Words:  12594  Groups:  133  Countries: 1


In [8]:
time.sleep(15)
ISM_wgwords3 = PrepareWordLists(datasets3, ISM_wgwords)

2022-04-02 16:38:31.178057
1
2022-04-02 16:38:39.849672
501
2022-04-02 16:38:49.304664
1001
2022-04-02 16:38:59.842255
1501
2022-04-02 16:39:10.544614
2001
2022-04-02 16:39:21.668222
2501
2022-04-02 16:39:35.838572
3001
2022-04-02 16:39:51.223821
3501
2022-04-02 16:40:08.054042
4001
2022-04-02 16:40:09.174759
Words:  10998  Groups:  118  Countries: 1


In [9]:
ISM_wgwords_all = ISM_wgwords1.append(ISM_wgwords2)
ISM_wgwords_all = ISM_wgwords_all.append(ISM_wgwords3)
## Build Indexed wordlist dataframe
ISM_wgwords_uniques = buildwordsindex(ISM_wgwords_all, 2, 100)

In [2]:
#with open('ISM_wgwords_uniques', 'wb') as f:
#    pickle.dump(ISM_wgwords_uniques, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('ISM_wgwords_uniques', 'rb') as f:
    ISM_wgwords_uniques = pickle.load(f)

In [11]:
len(ISM_wgwords_uniques['WORDS'].unique())

9776

## Transform SourceData into Numerics Structure Source Data

In [13]:
## Create Dataframe structrue with all the groups columns names
TrainingData = trainingstructuredata(ISM_wgwords_uniques)

In [14]:
## Create Training data Set
## Could be several process to reduce performance issues and finally consolidate on ISM_wgwords_all
square_values = True
ISM_Training_1 = training_dataset(ISM_test_set1, TrainingData, ISM_wgwords_uniques, square_values)

2022-04-02 16:41:43.065455
1
2022-04-02 16:41:43.291978
2
2022-04-02 16:41:43.630655
4
2022-04-02 16:41:44.356460
8
2022-04-02 16:41:45.837942
16
2022-04-02 16:41:48.627519
32
2022-04-02 16:41:54.356564
64
2022-04-02 16:42:05.449500
128
2022-04-02 16:42:28.111618
256
2022-04-02 16:43:15.822044
512
2022-04-02 16:44:48.949474
1024
2022-04-02 16:47:52.979967
2048
2022-04-02 16:54:19.056867
4096
Finished:  2022-04-02 16:57:05.460817  Errors: 774845


In [15]:
time.sleep(15)
ISM_Training_2 = training_dataset(ISM_test_set2, TrainingData, ISM_wgwords_uniques, square_values)

2022-04-02 16:57:30.747320
1
2022-04-02 16:57:30.947365
2
2022-04-02 16:57:31.354037
4
2022-04-02 16:57:32.057120
8
2022-04-02 16:57:33.474236
16
2022-04-02 16:57:36.311113
32
2022-04-02 16:57:42.064360
64
2022-04-02 16:57:53.675692
128
2022-04-02 16:58:19.145479
256
2022-04-02 16:59:06.568532
512
2022-04-02 17:00:40.640131
1024
2022-04-02 17:03:45.378789
2048
2022-04-02 17:10:02.049818
4096
Finished:  2022-04-02 17:12:53.880343  Errors: 774845


In [16]:
time.sleep(15)
ISM_Training_3 = training_dataset(ISM_test_set3, TrainingData, ISM_wgwords_uniques, square_values)

2022-04-02 17:13:19.172780
1
2022-04-02 17:13:19.398348
2
2022-04-02 17:13:19.762094
4
2022-04-02 17:13:20.472147
8
2022-04-02 17:13:21.907016
16
2022-04-02 17:13:24.705610
32
2022-04-02 17:13:30.268602
64
2022-04-02 17:13:41.448891
128
2022-04-02 17:14:05.050437
256
2022-04-02 17:14:51.324474
512
2022-04-02 17:16:24.024808
1024
2022-04-02 17:19:28.809790
2048
Finished:  2022-04-02 17:25:34.881662  Errors: 625115


In [19]:
ISM_Training_load = ISM_Training_1.append(ISM_Training_2)
ISM_Training_load = ISM_Training_load.append(ISM_Training_3)

## In Case of Multiple Data Sets, append all on ISM_Training_load
#ISM_Training_load = ISM_Training_1.append(ISM_Training_2)
#ISM_Training_load = ISM_Training_load.append(ISM_Training_3)

In [55]:
ISM_Training_load[ISM_Training_load['TICKETID']=='SR2665575']

,C-CEX-MX-EA-BASIS-PR-CX-L3,I-CEX-LATAM-AFST-CO-PR-MNT-L3,I-CEX-MX-BO-JDE-GSC-L1,I-CEX-NOAM-IT-GSC-L1,I-CEX-MY-SERVICEDESK-L1,C-CEX-GLOBAL-IT-AZU-CLD-MGM,I-CEX-EMEAA-BO-CS-APP-L3,I-CEX-EMEAA-BOCOM-SEC-L1,I-CEX-US-SERVICEDESK-L1,I-CEX-MX-OP-GINCO-GSC-L1,...,I-CEX-IN-IT-OAM-DTC-L2,C-CEX-AE-BO-GL-SEC-L2,I-CEX-MX-IT-SEC-CAB,I-CEX-MX-IT-SCCM-APP-L2,I-CEX-MX-CONTACTCENTER-GV-L2,I-CEX-MX-CONTACTCENTER-GV-L1,I-CEX-IN-IT-BURA-DTC-L2,COUNTRY,OWNERGROUP,TICKETID
2806,9,121,225,225,36,121,16,81,121,81,...,9,0,0,1,0,0,0,Mexico,I-CEX-MXBOSAP-SEC-APP-L3,SR2665575


## Create Training Data

In [51]:
split_test_size = 0.25
X_train, X_test, y_train, y_test, group_mapping = Create_training_data(ISM_Training_load, split_test_size)

with open('group_mapping', 'wb') as f:
    pickle.dump(group_mapping, f, protocol=pickle.HIGHEST_PROTOCOL)

In [53]:
group_mapping

,OWNERGROUP,MAP_VALUE
0,C-CEX-MX-EA-BASIS-PR-CX-L3,0
1,I-CEX-NOAM-IT-GSC-L1,1
2,I-CEX-MX-OP-GINCO-GSC-L1,2
3,I-CEX-MX-IT-SER-OS-L2,3
4,C-CEX-DE-IT-SAP-SEC-L2,4
...,...,...
150,I-CEX-IN-IT-MAILCOL-DTC-L2,150
151,C-CEX-HR-IT-CP-L2,151
152,I-CEX-GB-IT-FIN-APP-ENT-L2,152
153,I-CEX-PH-IT-IMAC-L2,153


## Create Models

In [43]:

## Dummy Model & Test
dummy_model = DummyClassifier(strategy='uniform').fit(X_train, y_train)
y_predicted = dummy_model.predict(X_test)
accuracy_score(y_test, y_predicted), precision_score(y_test, y_predicted, average='macro'), recall_score(y_test, y_predicted, average='macro')

C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0068415051311288486, 0.006442796805568306, 0.005942491478126676)

In [20]:
predictmodel =  xgb.XGBClassifier()
gs = GridSearchCV(predictmodel, {'n_estimators':range (80, 90, 10), 'max_depth':range (5, 10, 5)}, cv=5, n_jobs=-1, verbose=0, scoring = 'accuracy')
gs.fit(X_train, y_train)
gs.best_score_ , gs.best_params_

C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\mamoralessr\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mamoralessr\Anaconda3\lib\site-packages

[09:12:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.8108510638297872, {'max_depth': 5, 'n_estimators': 80})

In [44]:
##Result from GridSearch
## (0.7608003108003107, {'max_depth': 10, 'n_estimators': 86}) - Small number of records
## (0.7594017094017094, {'max_depth': 10, 'n_estimators': 90}) - Small number of records
## (0.8108510638297872, {'max_depth': 5, 'n_estimators': 80}) - Oct-Nov 14k records

#XGBClass_model = xgb.XGBClassifier(**gs.best_params_)
XGBClass_model = xgb.XGBClassifier({'max_depth': 5, 'n_estimators': 80})
XGBClass_model.fit(X_train,y_train)
y_predicted = XGBClass_model.predict(X_test)
accuracy_score(y_test, y_predicted), precision_score(y_test, y_predicted, average='macro'), recall_score(y_test, y_predicted, average='macro')

## Results from Training
## (0.8205571150939321, 0.6281368465483518, 0.5583273192811594) {'max_depth': 5, 'n_estimators': 80}) - Oct-Nov 14k records

C:\Users\mamoralessr\AppData\Roaming\Python\Python39\site-packages\xgboost\core.py:499: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\mamoralessr\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\User

[23:37:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.8332383124287344, 0.594639819139813, 0.5188617549890285)

In [45]:
## Revisar Resultados

validation_data = X_test 
groups_dic = {group_mapping.loc[idx, 'MAP_VALUE']: group_mapping.loc[idx, 'OWNERGROUP'] for idx in range(group_mapping.shape[0])}
validation_data['PREDICT'] = y_predicted
validation_data['REAL'] = y_test
validation_data['PREDICT'] = validation_data['PREDICT'].map(groups_dic)
validation_data['REAL'] = validation_data['REAL'].map(groups_dic)
validation_data[['PREDICT','REAL']].head(3)


,PREDICT,REAL
TICKETID,,
SR2665575,I-CEX-MXBOSAP-SEC-APP-L3,I-CEX-MXBOSAP-SEC-APP-L3
SR2639635,I-CEX-NOAM-IT-GSC-L1,I-CEX-NOAM-IT-GSC-L1
SR2654912,I-CEX-MX-PSD-LE-PA-L3,I-CEX-MX-PSD-LE-PA-L3


In [47]:
validation_data.to_excel(f"./Data/validation_data.xlsx")

In [52]:
with open('XGBClass_model', 'wb') as f:
    pickle.dump(XGBClass_model, f, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:

parameters = {'n_estimators': range(100, 200, 25),
              'criterion': ['gini', 'entropy'],
              'max_depth': list(range(1, 10))+[None],
              
              'max_features': ['auto', 'sqrt', 'log2']
             }
model = RandomForestClassifier()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


(0.7448936170212767,
 {'criterion': 'gini',
  'max_depth': None,
  'max_features': 'sqrt',
  'n_estimators': 150})

In [24]:
## Result from GridSearch
## (0.7448936170212767,  {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 150})

Forestmodel = RandomForestClassifier(**gs.best_params_)
Forestmodel.fit(X_train, y_train)
y_predicted = Forestmodel.predict(X_test)
accuracy_score(y_test, y_predicted), precision_score(y_test, y_predicted, average='macro'), recall_score(y_test, y_predicted, average='macro')

## Result from Training
## (0.7596631397106457, 0.508493199793195, 0.3771914430963358) {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt','n_estimators': 150})

C:\Users\MAMORA~1\AppData\Local\Temp/ipykernel_21188/2653838526.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  Forestmodel.fit(X_train, y_train)
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.7596631397106457, 0.508493199793195, 0.3771914430963358)

In [25]:
with open('Forestmodel', 'wb') as f:
    pickle.dump(Forestmodel, f, protocol=pickle.HIGHEST_PROTOCOL)